**Умова:**
    
Розглянемо дві різні умовні акції X і Y. Для кожної з них за допомогою статистичних досліджень були розраховані відносні
частоти очікуваних прибутків залежно від п'яти станів економічного середовища: значне піднесення(ЗП), незначне 
піднесення(НЗП), стагнація(С), незначний спад(НЗС), значний спад(ЗС). Ці дані занесені відповідно в останній рядок і 
останній стовпчик таблиці. Залежно від економічного середовища прогнозовані норми очікуваних прибутків у відсотках задані
відповідно у першому рядку та першому стовпчику таблиці. У клітинах на перетині X i Y проставлені ймовірності появи норм
відсотків значень (х, у). 
Необхідно:

1). обчислити очікувану норму прибутку по кожній з акцій;

2). оцінити ступінь ризику обох акцій за допомогою середньоквадратичного відхилення;

3). проаналізувати наявність зв'язку між акціями X та Y за допомогою коефіціентів коваріації то кореляції.

| **Y / X** |10      | 5      | 2      | 1      | -5     | P(Y)  |
| :---      | :----: | :----: | :----: | :----: | :----: | ---:  |
| **20**    | 0,01   | 0,03   | 0,02   | 0,03   |0,01    |**0,1**|
| **10**    | 0,03   | 0,09   | 0,06   | 0,09   |0,03    |**0,3**|
| **2**     | 0,02   | 0,06   | 0,04   | 0,06   |0,02    |**0,2**|
| **-2**    | 0,03   | 0,09   | 0,06   | 0,09   |0,03    |**0,3**|
| **-8**    | 0,01   | 0,03   | 0,02   | 0,03   |0,01    |**0,1**|
| **P(X)**  | **0,1**|**0,3** |**0,2** |**0,3** |**0,1** |**1**  |

In [35]:
import pandas as pd

df_x = pd.DataFrame({"X" : [10, 5, 2, 1, -5],
                     "P" : [0.1, 0.3, 0.2, 0.3, 0.1]})

df_y = pd.DataFrame({"Y" : [20, 10, 2, -2, -8],
                     "P" : [0.1, 0.3, 0.2, 0.3, 0.1]})

df_xy = pd.DataFrame([[0.01, 0.03, 0.02, 0.03, 0.01], 
                     [0.03, 0.09, 0.06, 0.09, 0.03],
                     [0.02, 0.06, 0.04, 0.06, 0.02],
                     [0.03, 0.09, 0.06, 0.09, 0.03],
                     [0.01, 0.03, 0.02, 0.03, 0.01]])

**Розв'язок:**


1. Знайдемо математичне очікування прибутку по кожній групі акцій:

In [36]:
df_x["M_X_i"] = df_x['X'] * df_x['P']
M_X = df_x["M_X_i"].sum()

df_y["M_Y_i"] = df_y['Y'] * df_y['P']
M_Y = df_y["M_Y_i"].sum()

print("M_X = ", round(M_X, 2))
print("M_Y = ", round(M_Y, 2))

M_X =  2.7
M_Y =  4.0


2. Знайдемо дисперсію по кожній групі акцій:

In [37]:
df_x["D_X_i"] = df_x["X"]**2 * df_x["P"]
D_X = df_x["D_X_i"].sum() - M_X**2

df_y["D_Y_i"] = df_y["Y"]**2 * df_y["P"]
D_Y = df_y["D_Y_i"].sum() - M_Y**2

print("D_X = ", round(D_X, 2))
print("D_Y = ", round(D_Y, 2))

D_X =  13.81
D_Y =  62.4


3. Знайдемо середньквадратичне відхилення:

In [38]:
Dlt_X = D_X**0.5
Dlt_Y = D_Y**0.5

print("Dlt_X = ", round(Dlt_X, 2))
print("Dlt_Y = ", round(Dlt_Y, 2))

Dlt_X =  3.72
Dlt_Y =  7.9


4. Знайдемо коефіцієнт коваріації за формулою:

$$ cov(X/Y) = M(XY) - MX * MY; $$

In [45]:

M_XY = 0
for i in df_xy:
    M_XY += sum(df_xy[i] * df_y["Y"] * df_x.at[i, 'X'])
print("M_XY = ", round(M_XY, 2))
    
cov_XY = M_XY - M_X * M_Y
print(f"cov_XY = {M_XY} - {round(M_X, 2)} * {round(M_Y, 2)} = ", round(cov_XY, 2))    

M_XY =  10.8
cov_XY = 10.8 - 2.7 * 4.0 =  0.0


Коефіцієнт кореляції розраховується за формулою:

$$ cor(X/Y) = \frac{cov(X/Y)}{\sqrt{DX}*\sqrt{DY}}; $$

тому з коефіцієнтом коваріації, що = 0, коефіцієнт кореляції також буде = 0.

**Це свідчить про відсутність корелювання між даними акціями.**

**Висновок:**

**За результатами даного дослідження, середньоквадратичне відхилення значно перевищує математичне очікування за обома 
групами акцій, що свідчить про вкрай високу ступінь ризику по роботі з обома пакетами.**